# Neural Network model for fluid dynamics

INTRODUCTION # TODO


As a general guideline for an ML/NN (Machine Learning/Neural Network) project, we follow the A. Géron checklist:

1. Frame the problem and look at the big picture. (In our case: Studying the flow in a channel)
2. Get the data. (As in: Acquiring velocity profiles from experiments)
3. Explore the data to gain insights. (See Section 2 - *Data Download and Inspection*)
4. Prepare the data to expose underlying patterns to Machine Learning algorithms. (Refer to Section 3 - *Downsampling the Database*)
5. Explore different models and shortlist the best ones. (See Section 4 - *Model 1: Binary Classification*)
6. Fine-tune your models and combine them for an optimized solution. (Exercise)
7. Present your solution. (Skipped, as it's not relevant for this lab class)
8. Launch, monitor, and maintain your system. (Skipped, as it's not relevant for this lab class)

## Installing Libraries

We begin by installing the necessary libraries to support our data manipulation, visualization, and deep learning modeling.

In [ ]:
%pip install numpy pandas scipy matplotlib

And now we import the necessary libraries

In [ ]:
import os  # Operating system-related functions
import pathlib  # Path manipulation and filesystem-related operations

import matplotlib.pyplot as plt  # Data visualization library
import numpy as np  # Numerical computing library
import pandas as pd  # Data manipulation and analysis library
import tensorflow as tf  # Deep learning framework for neural networks
from tensorflow.keras import Sequential, layers, losses, metrics, optimizers

## Data download and inspection

In this section, we will download the dataset and inspect its components.

### Download dataset files

In [ ]:
!wget https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/data/lab2/velprof-Re.csv
!wget https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/data/lab2/velprof-data.csv
!wget https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/data/lab2/velprof-space.csv

### Read Reynolds (the labels for our model)

We'll start by reading the Reynolds data, which serves as the labels for our model.

In [ ]:
data_Re = pd.read_csv("velprof-Re.csv", index_col=False)
data_Re.head()

### Read the data file (the features for our model)

Next, we'll read the data file containing the features for our model.

In [ ]:
data_v = pd.read_csv("velprof-data.csv", index_col=False)
data_v.head()

Let's also read another data file that contains information about space discretization, which, in our analogy, represents the probe's position.

In [ ]:
data_r = pd.read_csv("velprof-space.csv", index_col=False)
data_r.head()

We can also merge the two datasets, which include both labels and features (it can be useful later).

In [ ]:
data_all = pd.concat([data_v, data_Re], axis=1)

### Data Inspection and Visualization

In this section, we will perform the following tasks:
1. Plot a random velocity profile to visually examine the data.
2. Compute and create a correlation matrix plot to analyze the relationships between variables.

In [ ]:
# Create a plot of random data from the datasets
fig = plt.figure(figsize=(5, 3))
indx = np.random.randint(data_v.shape[0] - 1)

# Set the style of the plot to "seaborn-v0_8-paper"
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)

    # Plot the data and labels it with Re value
    ax.plot(
        data_r.iloc[indx, :],
        data_v.iloc[indx, 4:],
        labels=f"Re = {data_Re.iloc[indx].values[0]:1.2f}",
    )

    # Add a legend, labels, and titles to the plot
    ax.legend(loc="lower left")
    ax.set_xlabel("r (m)")
    ax.set_ylabel("u (m/s)")

Let's study the data and its correlation

In [ ]:
# Correlation Matrix Plot

# Define the number of bins and create a figure
Nbins = 50
fig = plt.figure(figsize=(6, 6))

# Define the labels for the correlation matrix
labels = [
    "Re(-)",
    "mu(Pas)",
    "rho(kg/m3)",
    "L(m)",
    "R(m)",
    "vel[0](m/s)",
    "vel[1](m/s)",
]
N = len(labels)

# Calculate the correlation matrix
corr_matrix = data_all.corr()

# Apply the "seaborn-v0_8-paper" style
with plt.style.context("seaborn-v0_8-paper"):
    grid = fig.add_gridspec(N, N, wspace=0.03, hspace=0.03)
    ax = []
    cmap = plt.get_cmap("Blues")

    for i in range(N):
        for j in range(N):
            ax.append(fig.add_subplot(grid[i, j]))

            # Plot scatter for lower triangle, correlation value for upper triangle, and histograms for diagonal
            if j < i:
                ax[-1].scatter(data_all[labels[j]], data_all[labels[i]], s=1, alpha=0.1)
            elif j > i:
                corr = corr_matrix.loc[labels[j], labels[i]]
                ax[-1].text(0.5, 0.5, f"{corr:1.3f}", ha="center", va="center")
                ax[-1].set_facecolor(cmap(np.abs(corr)))
                ax[-1].set_xticks([])
                ax[-1].set_yticks([])
            else:
                ax[-1].hist(data_all[labels[i]], bins=Nbins, alpha=0.5)

            # Set labels for x and y axes
            if j == 0 or i == N - 1:
                if j == 0 and i != N - 1:
                    ax[-1].set_ylabel(labels[i])
                    ax[-1].set_xticklabels([])
                elif i == N - 1 and j != 0:
                    ax[-1].set_xlabel(labels[j])
                    ax[-1].set_yticklabels([])
                else:
                    ax[-1].set_ylabel(labels[i])
                    ax[-1].set_xlabel(labels[j])

            # Remove tick labels
            ax[-1].set_xticklabels([])
            ax[-1].set_yticklabels([])

    # Set the title for the correlation matrix
    fig.suptitle("(Pearson) Correlation matrix")

# Show the plot
plt.show()

## Downsampling the database

The dataset contains too many data points (100,000), so we will reduce it to 5,000 by randomly selecting from the entire database.

In [ ]:
# Get the total number of data points in data_v
Nall = data_v.shape[0]

# Define the desired number of smaller data points to select
Nsmall = 5000

# Initialize a random number generator with a specific seed for reproducibility
rand_gen = np.random.default_rng(seed=1234)

# Generate a random sample of indices without replacement from the range [0, Nall)
# This will be used to select a subset of data_v and data_Re
indx = rand_gen.choice(np.arange(Nall), size=Nsmall, replace=False)

# Create smaller subsets of data_v and data_Re based on the randomly selected indices
data_v_small = data_v.iloc[indx, :]
data_Re_small = data_Re.iloc[indx, :]

let's store it as `.csv` (comma-separated values)

In [ ]:
data_v_small.to_csv("small-data.csv", index=False)
data_Re_small.to_csv("small-Re.csv", index=False)

## Model 1: Binary Classification

In this section, we will employ our Neural-Network model for binary classification with the primary goal of distinguishing between turbulent and non-turbulent conditions. It's essential to clarify that, within the context of our database, "non-turbulent" encompasses both laminar and transitional conditions. Please note that for transitional conditions, the velocity profile results are obtained through interpolation, which may exhibit empirical inaccuracy.

### Setup database

We load the features and the labels of our first model.

In [ ]:
features = pd.read_csv("small-data.csv", index_col=False).iloc[
    :, 4:
]  # Note: we drop the first 4 columns to study only the velocity profile
labels = pd.read_csv("small-Re.csv", index_col=False)

Display the first few rows of the features.

In [ ]:
features.head()

Display the first few rows of the labels.

In [ ]:
labels.head()

Next, we will convert the labels from numerical (`float`) to boolean (`bool`) using the following criteria:

* `True` when the flow is turbulent ($Re \ge 10^4$).
* `False` when the flow is non-turbulent, which includes both laminar and transitional regimes ($Re < 10^4$).

In [ ]:
labels_Re = labels.pop("Re(-)")
labels["Turbolent"] = labels_Re >= 1e4

Display the first few rows of the updated labels.

In [ ]:
labels.head()

To create a training and test dataset, we perform an 80/20 split. 

The purpose of this split is to reserve a portion of the data for testing the model's performance while ensuring that the two sets have a statistically similar distribution of features and labels. Since the database is already shuffled, we can conveniently take the first 1000 data points as the test set, given the total dataset size of 5000 items.

In [ ]:
# Splitting the dataset into training and test sets
# Training set (80% of the data)
labels_train = labels.iloc[1000:, :]
features_train = features.iloc[1000:, :]
labels_Re_train = labels_Re.iloc[1000:]

# Test set (20% of the data)
labels_test = labels.iloc[:1000, :]
features_test = features.iloc[:1000, :]
labels_Re_test = labels_Re.iloc[:1000]

### Storing

We will store the datasets and labels in a structured folder for future use.

In [ ]:
pathlib.Path("model_1").mkdir(parents=True, exist_ok=True)  # Make a folder
# Make train and test subfolders
pathlib.Path(os.path.join("model_1", "train")).mkdir(exist_ok=True)
pathlib.Path(os.path.join("model_1", "test")).mkdir(exist_ok=True)
# Storing
labels_train.to_csv(os.path.join("model_1", "train", "labels.csv"), index=False)
labels_test.to_csv(os.path.join("model_1", "test", "labels.csv"), index=False)
features_train.to_csv(os.path.join("model_1", "train", "features.csv"), index=False)
features_test.to_csv(os.path.join("model_1", "test", "features.csv"), index=False)

### First Try

#### Building the Model with Keras and TensorFlow

Now we define a function to create our neural network model using Keras and TensorFlow. This involves designing the layers, specifying their connections, and defining their behavior, including activation functions, regularization, dropout, and more.

In a neural network, a *layer* serves as a fundamental building block responsible for processing data and extracting features. Layers are combined to form the architecture of the neural network. Each layer consists of one or more "neurons" (also known as "nodes" or "units").

For our first model, we need to include the following types of layers:

* **Input Layer**: This is the initial layer that receives raw input data. Its primary role is to pass the data to subsequent layers. The input layer typically has one neuron for each feature present in the input data.

* **Hidden Layers**: These intermediate layers sit between the input and output layers. Hidden layers perform complex transformations on the data, enabling the network to learn and extract features from the input. Each neuron in a hidden layer receives input from multiple neurons in the previous layer.

* **Output Layer**: The final layer in the neural network is responsible for producing the model's predictions. The number of neurons in the output layer depends on the nature of the problem being addressed. For binary classification (True/False or 0/1), a single neuron is common in the output layer. In multiclass classification or regression tasks, the number of output neurons can vary.

In addition to layers, we also introduce the concept of an *activation function*. An activation function is a critical element within each neuron of a neural network. It dictates how the output of a neuron is calculated based on its input. The activation function introduces non-linearity into the model, allowing it to learn complex patterns and relationships within the data.

Common activation functions include:

* **ReLU (Rectified Linear Unit)**: ReLU is one of the most widely used activation functions. It returns the input value if it's positive and zero if it's negative. Mathematically, it can be represented as \(f(x) = \max(0, x)\). ReLU is effective in training deep networks and addressing the vanishing gradient problem.

* **Sigmoid**: The sigmoid activation function is commonly used in the output layer of binary classification models. It squashes the output into a range between 0 and 1, which can be interpreted as a probability. Mathematically, it's expressed as \(f(x) = \frac{1}{1 + e^{-x}}\).

* **Tanh (Hyperbolic Tangent)**: Tanh is similar to the sigmoid function but maps values to the range between -1 and 1. It is often used in hidden layers. Mathematically, it's defined as \(f(x) = \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}\).

The choice of the activation function depends on the specific problem and the architecture of the network. Each activation function has its own strengths and weaknesses, and selecting the right one is a crucial part of the network design process.

In our case, we use the `relu` (Rectified Linear Unit) activation function in the first hidden layer and a smoother function like `sigmoid` for the output layer.


In [ ]:
def build_model_1(n_cols: int) -> tf.keras.models.Sequential:
    """
    Build a the first possible architecture for our neural network model.

    Args:
        n_cols (int): Number of input features.

    Returns:
        tf.keras.models.Sequential: A Keras Sequential model.
    """
    model = Sequential(
        [
            # Input layer with the specified input shape
            layers.InputLayer(input_shape=(n_cols,), name="input_layer"),
            # Add the first hidden layer with 64 perceptron and ReLU activation
            layers.Dense(64, activation="relu", name="hidden_layer_1"),
            # Add the output layer with a single perceptron (we expect a True/False answer) and sigmoid activation
            layers.Dense(1, activation="sigmoid", name="output_layer"),
        ]
    )
    return model

Let's call the function to build our model.

In [ ]:
model = build_model_1(features.shape[1])

#### Model Summary and Structure Visualization

Let's examine the summary of our compiled model and visualize its architecture.

Given that the input data has 50 features and the first hidden layer consists of 64 neurons (each with weights $w$ and biases $b$), we have the following:

- Number of parameters in the first hidden layer ($\mathbf{w}_{h1}$): $50 \times 64 = 3200$
- Number of parameters in the first hidden layer biases ($\mathbf{b}_{h1}$): $1 \times 64 = 64$

In addition, considering the output layer:

- Number of parameters in the output layer weights ($\mathbf{w}_{o}$): $64 \times 1 = 64$
- Number of parameters in the output layer biases ($\mathbf{b}_{o}$): $64 \times 1 = 1$

This results in a total of 3329 degrees of freedom (dofs) within our model.

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model=model, rankdir="LR", dpi=72, show_shapes=True, show_layer_activations=True
)

#### Model Compilation

Now we will compile our neural network model. Model compilation involves defining key components, such as the loss function, optimizer, learning rate, and evaluation metrics.

##### Loss Function

For our binary classification task, we use the Binary Cross-Entropy loss function. The *Binary Cross-Entropy* loss is calculated as:

$\text{Binary Cross-Entropy Loss} = -\frac{1}{N}\sum_{i=1}^{N}\left(y_i \log(p_i) + (1 - y_i) \log(1 - p_i)\right) $

where $N$ is the number of samples, $y_i$ represents the true labels, and $p_i$ is the predicted probability of the positive class.

In [ ]:
loss = losses.BinaryCrossentropy(from_logits=True)

##### Optimizer

We use the *Adam* optimizer, a popular choice for training neural networks. The [Adam optimization algorithm](https://doi.org/10.48550/arXiv.1412.6980) is a neural network-specific adaptation of the [Stochastic Gradient Descent (SGD)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) method.

In [ ]:
optimizer = optimizers.Adam(learning_rate=1e-2, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

##### Metrics

Metrics are functions needed to measure the behavior of our model. There are many to choose from depending on the task of the model. For our case:

- **Accuracy**: This metric measures the overall classification accuracy of the model. It is calculated as the ratio of correct predictions to the total number of samples.

- **Binary Accuracy**: It's a specific metric for binary classification tasks. We use a threshold of 0.5 to determine binary predictions. Binary accuracy is computed as:

$ \text{Accuracy} = \frac{\text{Number of Correct Predictions}}{\text{Total Number of Predictions}} $

In [ ]:
metrics = [
    metrics.Accuracy(),
    metrics.BinaryAccuracy(threshold=0.5),
]


##### Compilation

Finally, we compile the model by specifying the optimizer, loss function, and metrics.

In [ ]:
model.compile(
    optimizer,
    loss,
    metrics,
)

#### Training
In training, we define two key parameters:

In [ ]:
batch_size = 512
epochs = 100

* **Batch Size**: It specifies the number of training examples used in each iteration. A smaller batch size updates the model more frequently, while a larger one may improve training efficiency but requiring more volatile memory (RAM).

* **Epochs**: Each epoch represents one pass through the entire training dataset. It controls how many times the model iterates over the data, influencing convergence and potential overfitting.

Let's (finally) start the training process.

In [ ]:
history = model.fit(
    np.array(features_train),  # before to feed the data we convert it into an array
    np.array(labels_train).astype("float"),
    batch_size,
    epochs,
    validation_data=(
        np.array(features_test),
        np.array(labels_test).astype("float"),
    ),  # test-set
    verbose=1,  # 0 = silent, 1 = progress bar, 2 = one line per epoch
)

#### Plot Training Progress

Now, we can plot the training progress.

In [ ]:
# Plot traing loop
train_binary_accuracy = np.array(history.history["binary_accuracy"])
test_binary_accuracy = np.array(history.history["val_binary_accuracy"])
train_loss = np.array(history.history["loss"])
test_loss = np.array(history.history["val_loss"])


epochs_i = np.arange(1, train_loss.shape[0] + 1)

In [ ]:
fig = plt.figure(figsize=(6, 3.3))
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)
    ax_twin = ax.twinx()
    a1 = ax.plot(
        epochs_i,
        train_binary_accuracy * 100,
        color="k",
        ls="-",
        labels="Binary accuracy (train)",
    )
    a2 = ax.plot(
        epochs_i,
        test_binary_accuracy * 100,
        color="k",
        ls="--",
        labels="Binary accuracy (test)",
    )
    l1 = ax_twin.plot(epochs_i, train_loss, color="r", ls="-", labels="Loss (train)")
    l2 = ax_twin.plot(epochs_i, test_loss, color="r", ls="--", labels="Loss (test)")
    ax.set_xlabel("Epochs [-]")
    ax.set_ylabel("Binary accuracy [%]")
    ax_twin.set_ylabel("Loss [-]")
    ax_twin.legend(
        a1 + a2 + l1 + l2,
        [
            "Binary accuracy (train)",
            "Binary accuracy (test)",
            "Loss (train)",
            "Loss (test)",
        ],
        loc="center right",
    )
plt.show()

#### Model result analysis

Let's compare the model's predictions with the Reynolds number.

In [ ]:
# Predictions are generated using the model over the test set,
# which the model has never seen during training.
predictions = model.predict(np.array(features_test))

In [ ]:
fig = plt.figure(figsize=(7, 3.3))
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)
    axins = ax.inset_axes([0.3, 0.3, 0.65, 0.54])
    ax.scatter(labels_Re_test, predictions, s=7, alpha=0.4)
    axins.scatter(labels_Re_test, predictions, s=7, alpha=0.4)
    axins.set_xlim([0, 12000])
    ax.indicate_inset_zoom(axins, edgecolor="black")
    ax.axvline(2e3, lw=1, color="k", ls=":")
    ax.axvline(1e4, lw=1, color="k", ls=":")
    ax.set_xlabel("Re [-]")
    ax.set_ylabel("Turbulence probability [-]")

In the following code cell, we define functions for deeper analysis, which we can use also later in the project.

In [ ]:
def compute_confusion_matrix(labels, predicted):
    """
    Compute the confusion matrix and return its elements along with the indices of TP, TN, FP, and FN.

    Args:
        labels (array-like): The actual binary classification labels (0 or 1).
        predicted (array-like): The predicted binary classification labels (0 or 1).

    Returns:
        numpy.ndarray: The 2x2 confusion matrix.
        list: Indices of True Positives (TP)
        list: Indices of True Negatives (TN)
        list: Indices of False Positives (FP)
        list: Indices of False Negatives (FN)
    """
    if len(labels) != len(predicted):
        raise ValueError("Input arrays must have the same length.")

    confusion_matrix = np.zeros((2, 2), dtype=int)
    TP_indices, TN_indices, FP_indices, FN_indices = [], [], [], []

    for i, (l, p) in enumerate(zip(labels, predicted)):
        confusion_matrix[l][p] += 1
        if l == 1 and p == 1:
            TP_indices.append(i)
        elif l == 0 and p == 0:
            TN_indices.append(i)
        elif l == 0 and p == 1:
            FP_indices.append(i)
        elif l == 1 and p == 0:
            FN_indices.append(i)

    return confusion_matrix, TP_indices, TN_indices, FP_indices, FN_indices

def plot_conf_matrix(ax, confusion_matrix, class_names, cmap, title="Confusion Matrix"):
    """
    Plot the confusion matrix with percentages.

    Args:
        ax: Matplotlib axes to plot the matrix.
        confusion_matrix (numpy.ndarray): The confusion matrix.
        class_names: Class names for labeling.
        cmap: Colormap for the plot.
        title (str): Title for the plot.
    """
    conf_matrix_perc = (confusion_matrix.T / confusion_matrix.sum(axis=1)).T * 100.0
    cm = ax.imshow(
        conf_matrix_perc, interpolation="nearest", cmap=cmap, vmax=100.0, vmin=0.0
    )
    ax.set_title(title)
    plt.colorbar(cm)

    tick_marks = np.arange(len(class_names))
    ax.set_xticks(tick_marks, class_names)
    ax.set_yticks(tick_marks, class_names)
    ax.set_yticklabels(class_names, rotation=90, ha="right", va="center")
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            ax.text(
                j,
                i,
                f"{conf_matrix_perc[i, j]:3.2f} %",
                horizontalalignment="center",
                color="k",
            )

    ax.set_ylabel("True labels")
    ax.set_xlabel("Predicted labels")

def plot_velocity_conf_matrix(axs, cm_indices, velocity, class_names, title):
    """
    Plot velocity profiles for TP, TN, FP, and FN.

    Args:
        axs: Matplotlib axes for subplots.
        cm_indices: Confusion matrix indices.
        velocity: Array of velocity profiles.
        class_names: Class names for labeling.
        title (str): Title for the plot.
    """
    velocity = np.asarray(velocity)
    r = np.linspace(0, 1, velocity.shape[1])
    for i in range(2):
        for j in range(2):
            for k in cm_indices[i][j]:
                axs[i][j].plot(r, velocity[k, :])
            axs[i][j].set_yticks([np.mean(axs[i][j].get_ylim())], [class_names[i]])
            axs[i][j].set_xticks([np.mean(axs[i][j].get_xlim())], [class_names[j])
            axs[i][j].set_yticklabels(
                [class_names[i]], rotation=90, ha="right", va="center"
            )
    ax.set_ylabel("True labels")
    axs[0][0].set_ylabel("True labels")
    axs[1][1].set_xlabel("Predicted labels")
    axs[0][1].set_xticklabels([])
    axs[0][1].set_yticklabels([])
    axs[1][1].set_yticklabels([])
    axs[0][0].yaxis.set_label_coords(-0.22, 0.0, transform=axs[0][0].transAxes)
    axs[1][1].xaxis.set_label_coords(0.0, -0.22, transform=axs[1][1].transAxes)
    axs[0][0].set_title(title, loc="right", fontdict={"ha": "center"})

Now, let's compute the confusion matrix and plot it.

In [ ]:
conf_matrix, i_TP, i_TN, i_FP, i_FN = compute_confusion_matrix(
    np.squeeze(np.array(labels_test).astype(int)),
    np.squeeze(predictions > 0.5).astype(int),
)

fig = plt.figure(figsize=(14, 3.6))
with plt.style.context("seaborn-v0_8-paper"):
    grid = fig.add_gridspec(2, 8, wspace=0.0, hspace=0.0)
    ax_cm = fig.add_subplot(grid[:, :2])
    plot_conf_matrix(ax_cm, conf_matrix, ["Laminar", "Turbulent"], "summer")
    ax_cm_p = []
    for i in range(2):
        ax_ = []
        for j in range(2):
            ax_.append(fig.add_subplot(grid[i, j + 3]))
        ax_cm_p.append(ax_)

    ax_cm_pn = []
    for i in range(2):
        ax_ = []
        for j in range(2):
            ax_.append(fig.add_subplot(grid[i, j + 6]))
        ax_cm_pn.append(ax_)

    cm_i = [[i_TN, i_FP], [i_FN, i_TP]]
    plot_velocity_conf_matrix(
        ax_cm_p, cm_i, features_test, ["Laminar", "Turbulent"], "Veloecity profiles"
    )
    plot_velocity_conf_matrix(
        ax_cm_pn,
        cm_i,
        (features_test.T / features_test.max(axis=1).values).T,
        ["Laminar", "Turbulent"],
        "Veloecity profiles normalized",
    )


plt.show()

### What if we increse the model depth

We are going to add a hiden layer

In [ ]:
def build_model_1_deeper(n_cols):
    model = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(n_cols,), name="in"),
            tf.keras.layers.Dense(64, activation="relu", name="h1"),
            tf.keras.layers.Dense(64, activation="relu", name="h2"),
            tf.keras.layers.Dense(1, activation="sigmoid", name="out"),
        ]
    )
    return model

Let us call the function and build the model

In [ ]:
model_v1 = build_model_1_deeper(features.shape[1])

And let us see if if is all in order

In [ ]:
model_v1.summary()

In [ ]:
tf.keras.utils.plot_model(
    model=model_v1, rankdir="LR", dpi=72, show_shapes=True, show_layer_activations=True
)

Now we compile the model

In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=1e-2, beta_1=0.9, beta_2=0.999, epsilon=1e-08
)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [
    tf.keras.metrics.Accuracy(),
    tf.keras.metrics.BinaryAccuracy(threshold=0.5),
]

model_v1.compile(
    optimizer,
    loss,
    metrics,
)

Training

In [ ]:
batch_size = 512
epochs = 100

In [ ]:
history_v1 = model_v1.fit(
    np.array(features_train),  # before to feed the data we convert it into an array
    np.array(labels_train).astype("float"),
    batch_size,
    epochs,
    validation_data=(
        np.array(features_test),
        np.array(labels_test).astype("float"),
    ),  # test-set
    verbose=1,  # 0 = silent, 1 = progress bar, 2 = one line per epoch
)

Now we plot the training

In [ ]:
# Plot traing loop
train_binary_accuracy_v1 = np.array(history_v1.history["binary_accuracy"])
test_binary_accuracy_v1 = np.array(history_v1.history["val_binary_accuracy"])
train_loss_v1 = np.array(history_v1.history["loss"])
test_loss_v1 = np.array(history_v1.history["val_loss"])


epochs_i = np.arange(1, train_loss_v1.shape[0] + 1)

In [ ]:
fig = plt.figure(figsize=(6, 3.3))
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)
    ax_twin = ax.twinx()
    a1 = ax.plot(
        epochs_i,
        train_binary_accuracy_v1 * 100,
        color="k",
        ls="-",
        labels="Binary accuracy (train)",
    )
    a2 = ax.plot(
        epochs_i,
        test_binary_accuracy_v1 * 100,
        color="k",
        ls="--",
        labels="Binary accuracy (test)",
    )
    l1 = ax_twin.plot(epochs_i, train_loss_v1, color="r", ls="-", labels="Loss (train)")
    l2 = ax_twin.plot(epochs_i, test_loss_v1, color="r", ls="--", labels="Loss (test)")
    ax.set_xlabel("Epochs [-]")
    ax.set_ylabel("Binary accuracy [%]")
    ax_twin.set_ylabel("Loss [-]")
    ax_twin.legend(
        a1 + a2 + l1 + l2,
        [
            "Binary accuracy (train)",
            "Binary accuracy (test)",
            "Loss (train)",
            "Loss (test)",
        ],
        loc="center right",
    )
plt.show()

Now we compere the prediction respect the Reynolds number

In [ ]:
predictions_v1 = model_v1.predict(np.array(features_test))

In [ ]:
fig = plt.figure(figsize=(7, 3.3))
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)
    axins = ax.inset_axes([0.3, 0.3, 0.65, 0.54])
    ax.scatter(labels_Re_test, predictions_v1, s=7, alpha=0.4)
    axins.scatter(labels_Re_test, predictions_v1, s=7, alpha=0.4)
    axins.set_xlim([0, 12000])
    ax.indicate_inset_zoom(axins, edgecolor="black")
    ax.axvline(2e3, lw=1, color="k", ls=":")
    ax.axvline(1e4, lw=1, color="k", ls=":")
    ax.set_xlabel("Re [-]")
    ax.set_ylabel("Turbulence probability [-]")

In [ ]:
conf_matrix, i_TP, i_TN, i_FP, i_FN = compute_confusion_matrix(
    np.squeeze(np.array(labels_test).astype(int)),
    np.squeeze(predictions_v1 > 0.5).astype(int),
)

fig = plt.figure(figsize=(14, 3.6))
with plt.style.context("seaborn-v0_8-paper"):
    grid = fig.add_gridspec(2, 8, wspace=0.0, hspace=0.0)
    ax_cm = fig.add_subplot(grid[:, :2])
    plot_conf_matrix(ax_cm, conf_matrix, ["Laminar", "Turbulent"], "summer")
    ax_cm_p = []
    for i in range(2):
        ax_ = []
        for j in range(2):
            ax_.append(fig.add_subplot(grid[i, j + 3]))
        ax_cm_p.append(ax_)

    ax_cm_pn = []
    for i in range(2):
        ax_ = []
        for j in range(2):
            ax_.append(fig.add_subplot(grid[i, j + 6]))
        ax_cm_pn.append(ax_)

    cm_i = [[i_TN, i_FP], [i_FN, i_TP]]
    plot_velocity_conf_matrix(
        ax_cm_p, cm_i, features_test, ["Laminar", "Turbulent"], "Veloecity profiles"
    )
    plot_velocity_conf_matrix(
        ax_cm_pn,
        cm_i,
        (features_test.T / features_test.max(axis=1).values).T,
        ["Laminar", "Turbulent"],
        "Veloecity profiles normalized",
    )


plt.show()

### What if we increade the database

Let's build a medium size database with 40'000 points



In [ ]:
Nall = data_v.shape[0]
Nsmall = 40000

rand_gen = np.random.default_rng(seed=1234)
indx = rand_gen.choice(np.arange(Nall), size=Nsmall, replace=False)

data_v_medium = data_v.iloc[indx, :]
data_Re_medium = data_Re.iloc[indx, :]

let's store it

In [ ]:
data_v_medium.to_csv("medium-data.csv", index=False)
data_Re_medium.to_csv("medium-Re.csv", index=False)

as before we reload the data for the *features* and the *labels* of our model, and split it in training and test set



In [ ]:
features_v2 = pd.read_csv("medium-data.csv", index_col=False).iloc[
    :, 4:
]  # note: we drop the first 4 colomns to study only the velocity profile
labels_v2 = pd.read_csv("medium-Re.csv", index_col=False)

In [ ]:
labels_Re_v2 = labels_v2.pop("Re(-)")
labels_v2["Turbolent"] = labels_Re_v2 >= 1e4

In [ ]:
labels_train_v2 = labels_v2.iloc[8000:, :]
labels_test_v2 = labels_v2.iloc[:8000, :]
features_train_v2 = features_v2.iloc[8000:, :]
features_test_v2 = features_v2.iloc[:8000, :]
# the Re number will be useful later
labels_Re_train_v2 = labels_Re_v2.iloc[8000:]
labels_Re_test_v2 = labels_Re_v2.iloc[:8000]

Storing

In [ ]:
pathlib.Path("model_1").mkdir(parents=True, exist_ok=True)  # make a folder
# make train and test subfolder
pathlib.Path(os.path.join("model_1", "train")).mkdir(exist_ok=True)
pathlib.Path(os.path.join("model_1", "test")).mkdir(exist_ok=True)
# storin
labels_train.to_csv(os.path.join("model_1", "train", "labels_v2.csv"), index=False)
labels_test.to_csv(os.path.join("model_1", "test", "labels_v2.csv"), index=False)
features_train.to_csv(os.path.join("model_1", "train", "features_v2.csv"), index=False)
features_test.to_csv(os.path.join("model_1", "test", "features_v2.csv"), index=False)

We now build the intila version of our model and compile it

In [ ]:
model_v2 = build_model_1(features_v2.shape[1])

In [ ]:
model_v2.summary()

In [ ]:
tf.keras.utils.plot_model(
    model=model_v2, rankdir="LR", dpi=72, show_shapes=True, show_layer_activations=True
)

In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=1e-2, beta_1=0.9, beta_2=0.999, epsilon=1e-08
)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [
    tf.keras.metrics.Accuracy(),
    tf.keras.metrics.BinaryAccuracy(threshold=0.5),
]

model_v2.compile(
    optimizer,
    loss,
    metrics,
)

Training

In [ ]:
batch_size = 512
epochs = 100

In [ ]:
history_v2 = model_v2.fit(
    np.array(features_train_v2),  # before to feed the data we convert it into an array
    np.array(labels_train_v2).astype("float"),
    batch_size,
    epochs,
    validation_data=(
        np.array(features_test_v2),
        np.array(labels_test_v2).astype("float"),
    ),  # test-set
    verbose=1,  # 0 = silent, 1 = progress bar, 2 = one line per epoch
)

Now we perfome the usual analisys

In [ ]:
# Plot traing loop
train_binary_accuracy_v2 = np.array(history_v2.history["binary_accuracy"])
test_binary_accuracy_v2 = np.array(history_v2.history["val_binary_accuracy"])
train_loss_v2 = np.array(history_v2.history["loss"])
test_loss_v2 = np.array(history_v2.history["val_loss"])


epochs_i = np.arange(1, train_loss_v2.shape[0] + 1)

In [ ]:
fig = plt.figure(figsize=(6, 3.3))
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)
    ax_twin = ax.twinx()
    a1 = ax.plot(
        epochs_i,
        train_binary_accuracy_v2 * 100,
        color="k",
        ls="-",
        labels="Binary accuracy (train)",
    )
    a2 = ax.plot(
        epochs_i,
        test_binary_accuracy_v2 * 100,
        color="k",
        ls="--",
        labels="Binary accuracy (test)",
    )
    l1 = ax_twin.plot(epochs_i, train_loss_v2, color="r", ls="-", labels="Loss (train)")
    l2 = ax_twin.plot(epochs_i, test_loss_v2, color="r", ls="--", labels="Loss (test)")
    ax.set_xlabel("Epochs [-]")
    ax.set_ylabel("Binary accuracy [%]")
    ax_twin.set_ylabel("Loss [-]")
    ax_twin.legend(
        a1 + a2 + l1 + l2,
        [
            "Binary accuracy (train)",
            "Binary accuracy (test)",
            "Loss (train)",
            "Loss (test)",
        ],
        loc="center right",
    )
plt.show()

In [ ]:
predictions_v2 = model.predict(np.array(features_test_v2))

In [ ]:
fig = plt.figure(figsize=(7, 3.3))
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)
    axins = ax.inset_axes([0.3, 0.3, 0.65, 0.54])
    ax.scatter(labels_Re_test_v2, predictions_v2, s=7, alpha=0.4)
    axins.scatter(labels_Re_test_v2, predictions_v2, s=7, alpha=0.4)
    axins.set_xlim([0, 12000])
    ax.indicate_inset_zoom(axins, edgecolor="black")
    ax.axvline(2e3, lw=1, color="k", ls=":")
    ax.axvline(1e4, lw=1, color="k", ls=":")
    ax.set_xlabel("Re [-]")
    ax.set_ylabel("Turbulence probability [-]")

Futher analysis

In [ ]:
conf_matrix, i_TP, i_TN, i_FP, i_FN = compute_confusion_matrix(
    np.squeeze(np.array(labels_test_v2).astype(int)),
    np.squeeze(predictions_v2 > 0.5).astype(int),
)

fig = plt.figure(figsize=(14, 3.6))
with plt.style.context("seaborn-v0_8-paper"):
    grid = fig.add_gridspec(2, 8, wspace=0.0, hspace=0.0)
    ax_cm = fig.add_subplot(grid[:, :2])
    plot_conf_matrix(
        ax_cm, conf_matrix, ["Laminar", "Turbulent"], "summer", title="CM Model v0"
    )
    ax_cm_p = []
    for i in range(2):
        ax_ = []
        for j in range(2):
            ax_.append(fig.add_subplot(grid[i, j + 3]))
        ax_cm_p.append(ax_)

    ax_cm_pn = []
    for i in range(2):
        ax_ = []
        for j in range(2):
            ax_.append(fig.add_subplot(grid[i, j + 6]))
        ax_cm_pn.append(ax_)

    cm_i = [[i_TN, i_FP], [i_FN, i_TP]]
    plot_velocity_conf_matrix(
        ax_cm_p, cm_i, features_test_v2, ["Laminar", "Turbulent"], "Veloecity profiles"
    )
    plot_velocity_conf_matrix(
        ax_cm_pn,
        cm_i,
        (features_test_v2.T / features_test_v2.max(axis=1).values).T,
        ["Laminar", "Turbulent"],
        "Veloecity profiles normalized",
    )

    plt.show()

### Conclusion

Let's compare the 3 setup

In [ ]:
fig = plt.figure(figsize=(6, 3.3))
with plt.style.context("seaborn-v0_8-paper"):
    ax = fig.add_subplot(111)
    ax_twin = ax.twinx()
    ax_twin.plot(np.nan, np.nan, color="k", ls=":", labels="Binary accuracy (train)")
    ax_twin.plot(np.nan, np.nan, color="k", ls="-", labels="Binary accuracy (test)")
    ax_twin.plot(np.nan, np.nan, color="r", ls=":", labels="Loss (train)")
    ax_twin.plot(np.nan, np.nan, color="r", ls="-", labels="Loss (test)")
    ax.plot(
        epochs_i,
        train_binary_accuracy * 100,
        color="k",
        ls=":",
        marker="o",
        markevery=5,
        mfc="none",
        mew=1,
    )
    ax.plot(
        epochs_i,
        train_binary_accuracy_v1 * 100,
        color="k",
        ls=":",
        marker="s",
        markevery=5,
        mfc="none",
        mew=1,
    )
    ax.plot(
        epochs_i,
        train_binary_accuracy_v2 * 100,
        color="k",
        ls=":",
        marker="^",
        markevery=5,
        mfc="none",
        mew=1,
    )
    ax.plot(
        epochs_i,
        test_binary_accuracy * 100,
        color="k",
        ls="-",
        marker="o",
        markevery=5,
        mfc="none",
        mew=1,
        labels="Model v0",
    )
    ax.plot(
        epochs_i,
        test_binary_accuracy_v1 * 100,
        color="k",
        ls="-",
        marker="s",
        markevery=5,
        mfc="none",
        mew=1,
        labels="Model v1",
    )
    ax.plot(
        epochs_i,
        test_binary_accuracy_v2 * 100,
        color="k",
        ls="-",
        marker="^",
        markevery=5,
        mfc="none",
        mew=1,
        labels="Model v2",
    )

    ax_twin.plot(
        epochs_i,
        train_loss,
        color="r",
        ls=":",
        marker="o",
        markevery=10,
        mfc="none",
        mew=1,
    )
    ax_twin.plot(
        epochs_i,
        train_loss_v1,
        color="r",
        ls=":",
        marker="s",
        markevery=10,
        mfc="none",
        mew=1,
    )
    ax_twin.plot(
        epochs_i,
        train_loss_v2,
        color="r",
        ls=":",
        marker="^",
        markevery=10,
        mfc="none",
        mew=1,
    )
    ax_twin.plot(
        epochs_i,
        test_loss,
        color="r",
        ls="-",
        marker="o",
        markevery=10,
        mfc="none",
        mew=1,
    )
    ax_twin.plot(
        epochs_i,
        test_loss_v1,
        color="r",
        ls="-",
        marker="s",
        markevery=10,
        mfc="none",
        mew=1,
    )
    ax_twin.plot(
        epochs_i,
        test_loss_v2,
        color="r",
        ls="-",
        marker="^",
        markevery=10,
        mfc="none",
        mew=1,
    )
    ax.set_xlabel("Epochs [-]")
    ax.set_ylabel("Binary accuracy [%]")
    ax.set_ylim([98, 100])
    ax_twin.set_ylabel("Loss [-]")
    ax_twin.legend(loc="upper left", bbox_to_anchor=(1.2, 1.0))
    ax.legend(loc="lower left", bbox_to_anchor=(1.2, 0.0))
    # ax_twin.legend(a1+a2+l1+l2, ['Binary accuracy (train)', 'Binary accuracy (test)', 'Loss (train)', 'Loss (test)'], loc='center right')
plt.show()

In [ ]:
conf_matrix, _, _, _, _ = compute_confusion_matrix(
    np.squeeze(np.array(labels_test).astype(int)),
    np.squeeze(predictions > 0.5).astype(int),
)
conf_matrix_v1, _, _, _, _ = compute_confusion_matrix(
    np.squeeze(np.array(labels_test).astype(int)),
    np.squeeze(predictions_v1 > 0.5).astype(int),
)
conf_matrix_v2, _, _, _, _ = compute_confusion_matrix(
    np.squeeze(np.array(labels_test_v2).astype(int)),
    np.squeeze(predictions_v2 > 0.5).astype(int),
)

fig = plt.figure(figsize=(14, 3.6))
with plt.style.context("seaborn-v0_8-paper"):
    grid = fig.add_gridspec(2, 8, wspace=0.0, hspace=0.0)
    ax_cm_0 = fig.add_subplot(grid[:, :2])
    ax_cm_1 = fig.add_subplot(grid[:, 3:5])
    ax_cm_2 = fig.add_subplot(grid[:, 6:])

    plot_conf_matrix(
        ax_cm_0, conf_matrix, ["Laminar", "Turbulent"], "summer", title="CM Model v0"
    )
    plot_conf_matrix(
        ax_cm_1, conf_matrix_v1, ["Laminar", "Turbulent"], "summer", title="CM Model v1"
    )
    plot_conf_matrix(
        ax_cm_2, conf_matrix_v2, ["Laminar", "Turbulent"], "summer", title="CM Model v2"
    )

    plt.show()